In [ ]:
from tensorflow.keras import Sequential, layers, losses,optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import initializers
from tensorflow.keras import metrics
from tensorflow.keras.applications import EfficientNetB7
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
directory='../input/flowerdata-module5/train/train'
directory2='../input/flowerdata-module5/test/test'
DATA_PATH='../input/flowerdata-module5/train'
CHECKPOINT_PATH = 'models/'
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.3)
batch=64


In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory,
    target_size=(128, 128),
    batch_size=batch,
    class_mode='categorical',
    subset='training',
    shuffle=True,
)

validation_generator = validation_datagen.flow_from_directory(
    directory,
    target_size=(128, 128),
    batch_size=batch,
    class_mode='categorical',
    subset='validation',
    shuffle=True
    )

In [ ]:
model = Sequential([
  EfficientNetB7(input_shape=(128, 128, 3), include_top=False,weights="imagenet"),
  layers.GlobalMaxPool2D(),
  layers.Dropout(0.5),
  #layers.Flatten(),
  layers.Dense(102,'softmax')
])


In [ ]:
initial_learning_rate = 0.001
model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer= optimizers.Adam(initial_learning_rate),
    metrics='accuracy'
)


In [ ]:
def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * np.exp(-k*epoch)

filepath='../input/flowerdata-module5'
save_freq = len(train_generator) // batch

callbacks_list = [
     tf.keras.callbacks.ModelCheckpoint(
     filepath,
     monitor='val_loss',
     verbose=1,
     save_best_only=True,
     mode='auto',
     save_freq=save_freq,
     options=None,
     initial_value_threshold=None,
       ),
     tf.keras.callbacks.LearningRateScheduler(
        lr_exp_decay,
        verbose=1)
                  ]


In [ ]:
model.fit(train_generator,
    validation_data = validation_generator,
    callbacks= callbacks_list,
    batch_size = batch,
    epochs=30)

In [ ]:
submission = pd.read_csv('../input/flowerdata-module5/sample_submission.csv')
print(f'Shape: {submission.shape}')
submission.head()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
test_ds = test_datagen.flow_from_dataframe(
    dataframe=submission,
    directory=directory2,
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=19812,
    target_size=(128, 128),
    batch_size=32
)

In [ ]:
prediction=predictions = model.predict(test_ds, verbose=1)
predictions = np.argmax(predictions, axis=-1)

In [ ]:

prediction

In [ ]:
label_map = (train_generator.class_indices) #получаем словарь соответствия меток классов и самих классов
label_map = dict((v,k) for k,v in label_map.items()) #меняем местами ключи и значения в этом словаре
predictions = [label_map[k] for k in predictions] #перекодируем наше предсказание
test_submission = submission.copy()
test_submission['Category'] = pd.Series(predictions)
test_submission.to_csv('1my_submission.csv', index=False)
print('Save submit')